# Finding water with neural nets

HIDA - Datathon 2020 Solution by

**Team neutrons_net**


In [1]:
import ipywidgets as widgets
import numpy as np
from PIL import Image, ImageFont, ImageDraw 
import io
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2
import pandas as pd
import sys
import random
sys.path.insert(0,'.')
import colorsys

from src.utils import keras_losses as loss

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Generate random colors:
def random_colors(N, bright=True):
    """
    Generate random colors.
    To get visually distinct colors, generate them in HSV space then
    convert to RGB.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

In [3]:
def setup_model(path='notebooks/models/unet2_weights.h5'):
    model = loss.make_model(image_size=(224, 224))
    #model.compile(loss=loss.BCE_dice, optimizer='sgd', metrics=['accuracy'])
    model.load_weights(path)
    return model

In [4]:
model = setup_model()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [5]:
def visualize(input_img, output_img,feature_legend_path='notebooks/models/features_legend.csv', seed = 42):
    # load original image and image with annotations
    original_image = input_img
    #image_annotations = Image.open(annotated_image_path).convert('RGB')
    image_annotations = Image.fromarray(output_img)
    # set annotated image to semi-transparent
    image_annotations.putalpha(alpha = 128)
    
    # create nice looking colors
    random.seed(seed)
    colors = random_colors(20)
    
    # transform image so one can work with it
    image_transformed = np.array(image_annotations)

    height = np.array(image_annotations).shape[0]
    width = np.array(image_annotations).shape[1]

    image_transformed = image_transformed.reshape(height*width, 4)

    # Make black pixels transparent:
    image_transformed = [[0, 0, 0, 0]  if v == [0, 0, 0, 128] else v for v in image_transformed.tolist()]

    # Resetting RGB ID's to nice colors:
    for i in range(1,21) :
        r = colors
        image_transformed = [[colors[i][0]*255, colors[i][1]*255, colors[i][2]*255, 128]  if v == [i, i, i, 128] else v for v in image_transformed]


    #Reshape and make it Image type again
    image_transformed = np.array(image_transformed, dtype = 'uint8')
    image_transformed = image_transformed.reshape(height, width, 4)
    
    #Put original image in the background and paste annotations over it

    img = Image.fromarray(image_transformed)
    background = original_image
    background.putalpha(alpha = 255)
    background.paste(img, (0, 0), img)
    
    output_image_without_writing = background
    
    # Calculate percentages
    rgb_ids = pd.read_csv(feature_legend_path)
    
    image_annotations = output_img
    image_annotations = np.array(image_annotations)
    height = np.array(image_annotations).shape[0]
    width = np.array(image_annotations).shape[1]

    image_for_counting = image_annotations.reshape(height*width, 3).tolist()
    
    rgb_ids['percentage'] = 0
    for i in range(1, 21):
        rgb_ids['percentage'][rgb_ids['id_rgb'] == i] = (image_for_counting.count([i, i, i])/(height*width))*100
    
    # Write percentages on image
    dataframe_for_writing = rgb_ids[rgb_ids['percentage'] > 0]
    
    #Select Arial as font
    title_font = ImageFont.truetype(r'notebooks/models/Roboto/Roboto-Regular.ttf', 20)

    for i in range(0, dataframe_for_writing.shape[0]):
        title_text = f"{dataframe_for_writing[' label'].iloc[i]}: {round(dataframe_for_writing['percentage'].iloc[i], 2)}%"

        image_editable = ImageDraw.Draw(output_image_without_writing)
        image_editable.text((15,15 + i*25), title_text, (0, 0, 0), font=title_font)
        
    output_image = output_image_without_writing
    
    return output_image

In [6]:
def img_inference(img, model):
    img = np.array(img)[None, :, :, :]
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    pre = model.predict(img)

    pre = np.argmax(pre, axis=-1).squeeze()
    
    return pre

In [7]:
def on_click_classify(change):
    img = Image.open(io.BytesIO(btn_upload.data[-1]))
    img = img.resize((224, 224))
    pred = img_inference(img, model)

    pred = np.uint8((np.stack([pred, pred, pred], axis=-1)))

    pred = visualize(img, pred)
    with image_pl: 
        display(img)

    with prediction_pl: 
        display(pred)


btn_upload = widgets.FileUpload()
image_pl = widgets.Output()
prediction_pl = widgets.Output()
lbl_pred = widgets.Label()
lbl_pred.value = "Select an image for Classification!"
btn_run = widgets.Button(description='Classify')
btn_run.on_click(on_click_classify)

In [8]:
widgets.VBox([btn_upload, btn_run])

In [9]:
widgets.HBox([image_pl, prediction_pl])